<a href="https://colab.research.google.com/github/zahraa57/Deep-Learning-CV/blob/main/Assignment_3_CIFAR_10_Transfer_Learning1_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For CIFAR-10 dataset:
- Add Data augmentation, as in notebook 5.2
- Add TL, as in noteboook 5.3
- Optional: Visualize layers as in notebook 5.4

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10 .load_data()

170508288/170498071 [==============================] - 13s 0us/step


In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
#dev_labels=to_categorical(dev_labels)
test_labels = to_categorical(test_labels)

In [ ]:
train_labels.shape

(50000, 10)

In [ ]:
test_labels.shape

(10000, 10)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)
datagen.fit(train_images)

In [ ]:
# for data_batch, labels_batch in datagen:
#     print('data batch shape:', data_batch.shape)
#     print('labels batch shape:', labels_batch.shape)
#     break

In [ ]:
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu', name='Layer_2'))
model.add(layers.Dense(128, activation='relu', name='Layer_3'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['acc'])


In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
history = model.fit(
      datagen.flow(train_images, train_labels, batch_size=32,
         subset='training'),
         steps_per_epoch=int(len(train_images) /32),
         validation_data=datagen.flow(train_images, train_labels,
         batch_size=32, subset='validation'),
         validation_steps=len(train_images)//32,
          epochs=5)


Epoch 1/5
1562/1562 [==============================] - 45s 29ms/step - loss: 2.3028 - acc: 0.0980 - val_loss: 2.3030 - val_acc: 0.0937


In [ ]:
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()